In [8]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l1,l1_l2,l2
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import os
from collections import Counter
import numpy as np
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.multiclass import unique_labels

# from vis.utils import utils
from tensorflow.keras.activations import elu,relu 

from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling1D,GlobalAveragePooling2D
from tensorflow.keras.layers import Embedding, Flatten, Input, add,concatenate

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
from neural_net import build_model,build_and_train
import hyperopt_optimize
import cv2


In [9]:
# img=cv2.imread("/home/kashraf/Journal_work_Fall2021/Audio_topomaps_June21/beta/cl4/sub_1_beta_3.png")
# img.shape

In [10]:
hyperspace= {
        "activation": "relu",
        "batch_size": 112.0,
        "conv_dropout_drop_proba": 0.23500786106647073,
        "conv_hiddn_units_mult": 1.2316507928963547,
        "conv_kernel_size": 3.0,
        "conv_pool_res_start_idx": 1.0,
        "fc_dropout_drop_proba": 0.28105705270734643,
        "fc_units_1_mult": 1.5121176390097313,
        "first_conv": 4,
        "l2_weight_reg_mult": 1.1411104782264416,
        "lr_rate_mult": 0.670345540313049,
        "nb_conv_pool_layers": 3,
        "one_more_fc": 0.7127527455372467,
        "optimizer": "SGD",
        "pooling_type": "avg",
        "res_conv_kernel_size": 2.0,
        "residual": 4.0,
        "use_BN": False
    }
model=build_model(hyperspace)


Hyperspace:
{'activation': 'relu', 'batch_size': 112.0, 'conv_dropout_drop_proba': 0.23500786106647073, 'conv_hiddn_units_mult': 1.2316507928963547, 'conv_kernel_size': 3.0, 'conv_pool_res_start_idx': 1.0, 'fc_dropout_drop_proba': 0.28105705270734643, 'fc_units_1_mult': 1.5121176390097313, 'first_conv': 4, 'l2_weight_reg_mult': 1.1411104782264416, 'lr_rate_mult': 0.670345540313049, 'nb_conv_pool_layers': 3, 'one_more_fc': 0.7127527455372467, 'optimizer': 'SGD', 'pooling_type': 'avg', 'res_conv_kernel_size': 2.0, 'residual': 4.0, 'use_BN': False}
0
49
(None, 224, 224, 16)
(None, 224, 224, 49)
(None, 112, 112, 49)
1
98
(None, 112, 112, 49)
(None, 112, 112, 98)
(None, 112, 112, 98)
(None, 56, 56, 98)
2
196
(None, 56, 56, 98)
(None, 56, 56, 196)
(None, 56, 56, 196)
(None, 28, 28, 196)
(None, 153664)
(None, 1512)
(None, 534)


In [11]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 224, 224, 16) 784         lambda_2[0][0]                   
__________________________________________________________________________________________________
conv2d_41 (Conv2D)              (None, 224, 224, 49) 7105        conv2d_40[0][0]                  
____________________________________________________________________________________________

In [12]:
# validation_gen=ImageDataGenerator()
# validation_path="/home/kashraf/Journal_work_Fall2021/GAN_topomap/topos_GAN+REAL/stack/train_test/test/"
# img_rows, img_cols = 224,224
# validation_generator = validation_gen.flow_from_directory(
#     validation_path,
#     color_mode="rgb",

#     target_size=(img_rows, img_cols),
#     batch_size=32,
#     class_mode='categorical',
#     shuffle=False)


### Training model for beta

In [ ]:
model, model_name, result, log_path=build_and_train(hype_space=hyperspace,save_best_weights=True)

In [16]:
validation_data_dir ="/media/kashraf/Elements/Journal_2022/Datasets/topos_GAN+REAL/alpha/train_test/test"
model_path="/home/kashraf/Journal_work_Fall2021/modelling/real_gan_models/model_weights_april/alpha_April_V1.h5"
model=tensorflow.keras.models.load_model(model_path)

validation_datagen = ImageDataGenerator()
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=64,
    class_mode='categorical',
    shuffle=False)
model.evaluate(validation_generator)


Found 13200 images belonging to 4 classes.
207/207 [==============================] - 30s 143ms/step - loss: 4.1411 - accuracy: 0.8917


[4.141085624694824, 0.8917424082756042]

In [ ]:
import tensorflow as tf
tf.keras.models.load_model("/home/kashraf/Journal_work_Fall2021/modelling/real_gan_models/model_weights_Feb22/Beta_Feb22.h5").summary()

In [ ]:
np.save("beta_history.np",result, allow_pickle=True)


In [ ]:
import json

# python dictionary with key value pairs

# create json object from dictionary
json = json.dumps(result)

# open file for writing, "w" 
f = open("Stack_history.json","w")

# write json object to file
f.write(json)

# close file
f.close()

In [ ]:
results_path="/home/kashraf/Journal_work_Fall2021/modelling/real_gan_models/thesis results and figures/"
history=result["history"]

print(history.keys())
# Plotting our loss charts
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure(figsize=(15,7))
plt.xticks(fontsize=13,weight="bold")
plt.yticks(fontsize=13,weight="bold")
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_loss_values, label='Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label='Training Loss')
plt.setp(line1, linewidth=3.0,marker = '+', markersize=10.0)
plt.setp(line2, linewidth=3.0,marker = '4', markersize=10.0)
plt.xlabel('Epochs',fontsize=20,weight="bold") 
plt.ylabel('Loss',fontsize=20,weight="bold")
plt.grid(True)
plt.xticks(fontsize=13,weight="bold")
plt.yticks(fontsize=13,weight="bold")
plt.legend(fontsize=15)
plt.savefig(results_path+'Stack_Loss.png', dpi = 500)
plt.show()


In [7]:
results_path="/home/kashraf/Journal_work_Fall2021/modelling/real_gan_models/thesis results and figures/"
history=result["history"]

print(history.keys())
# Plotting our loss charts
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure(figsize=(15,7))
plt.xticks(fontsize=13,weight="bold")
plt.yticks(fontsize=13,weight="bold")
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_acc_values, label='Validation/Test Accuracy')
line2 = plt.plot(epochs, acc_values , label='Training Accuracy')
plt.setp(line1, linewidth=3.0,marker = '+', markersize=10.0)
plt.setp(line2, linewidth=3.0,marker = '4', markersize=10.0)
plt.xlabel('Epochs',fontsize=20,weight="bold") 
plt.ylabel('Loss',fontsize=20,weight="bold")
plt.grid(True)
plt.xticks(fontsize=13,weight="bold")
plt.yticks(fontsize=13,weight="bold")
plt.legend(fontsize=15)
plt.savefig(results_path+'Stack_Accuracy.png', dpi = 500)
plt.show()


NameError: name 'result' is not defined

In [ ]:
import seaborn as sr
import pandas as pd

y_pred=np.argmax(model.predict(validation_generator), axis=1)
y_test=validation_generator.classes


# y_predict,distances1=  face_recognition(x_test_recog,T_distance)
# accuracy=accuracy_score(y_test_recog,y_predict)
class_names=["CL_1","CL_2","CL_3","CL_4"]
report=classification_report(y_test,y_pred,target_names=class_names)

conf=confusion_matrix(y_test,y_pred,normalize="true")
conf_df=pd.DataFrame(conf, index=class_names, columns=class_names)
# print("\nFace  accuracy =",accuracy)


In [ ]:
# print("\n Classification report: \n",report)
# fig=plt.figure(figsize=(15,10),)
sr.heatmap(conf_df,annot=True,cmap="Blues")
# sr.fontsize()
# sr.set(font_scale=0)
# plt.title("Confusion matrix")
plt.savefig(results_path+"Confusion matrix_beta.png")
# plt.xticks(fontsize=13,weight="bold")
# plt.yticks(fontsize=13,weight="bold")
plt.show()